# New Opening PopUpStore

## Check Connection to Idealista 

In [53]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
from getpass import getpass
from sqlalchemy import inspect
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import requests as r
import re
import geopy 


In [186]:
url = 'https://api.fotocasa.es/PropertySearch/Search?combinedLocationIds=724,14,28,173,0,28079,0,0,0&culture=es-ES&featureIds=130&hrefLangCultures=ca-ES%3Bes-ES%3Bde-DE%3Ben-GB&isMap=false&isNewConstruction=false&isNewConstructionPromotions=false&latitude=40.4096&longitude=-3.68624&pageNumber=1&platformId=1&sortOrderDesc=true&sortType=bumpdate&transactionTypeId=3&propertyTypeId=2'


In [187]:
r.get(url)

<Response [200]>

In [188]:
headers = """accept: application/json, text/plain, */*
accept-encoding: gzip, deflate, br
accept-language: es-ES,es;q=0.9,fr;q=0.8,en;q=0.7,pl;q=0.6
cache-control: no-cache
origin: https://www.fotocasa.es
pragma: no-cache
referer: https://www.fotocasa.es/
sec-ch-ua: "Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"
sec-ch-ua-mobile: ?0
sec-fetch-dest: empty
sec-fetch-mode: cors
sec-fetch-site: same-site
user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"""

In [189]:
headers=dict(i.split(': ') for i in headers.split('\n'))

In [190]:
resp = r.get(url, headers=headers)

In [191]:
resp

<Response [200]>

In [149]:
data = resp.json()

In [150]:
data.keys()

dict_keys(['breadcrumb', 'count', 'hrefLang', 'client', 'realEstates', 'adjacentRealEstates'])

In [94]:
from requests.exceptions import SSLError, Timeout, TooManyRedirects, RequestException
from contextlib import closing
from IPython.display import Markdown, display
import warnings

warnings.filterwarnings('ignore')


timeout_glob=0.05
verify_glob=True
allow_glob=False

def printmd(string):
    display(Markdown('**'+string+'**'))
    
def is_good_response(x):
    """
    Returns True if the response seems to be an HTML, False otherwise
    x is response
    """
    
    content_type=x.headers['Content-Type'].lower()
    
    return ((x.status_code==200 and content_type is not None and content_type.find('html')>1), x.status_code)

def log_error(e):
    print(e)
    
    
def status_handler(status):
    if status<300:
        print('Alles gut!')
    elif status>=400 and status<500:
        print('request has failed due to you mistake bro. Check the link, headers, server and whether you have an access to this page.')
    else:
        print('Yoooo, I have no clue what has happened. But you are stuck, since you didnt receive any output from server')
    return None


def adaptive_scraping(url, timeout=timeout_glob, verify=verify_glob):
    global timeout_glob
    global verify_glob
    global allow_glob
    import requests as r
    
    try:
        with closing(r.get(url, stream=True, timeout=timeout_glob, verify=verify_glob, allow_redirects=allow_glob)) as resp:
            test1=is_good_response(resp)
            print(resp)
            if test1[0]:
                return resp.content
            else:
                return status_handler(test1[1])
    except Timeout as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('We can increase response waiting time. Do you want to continue?')
        ans=input('\n[Yes|No]\t')
        if ans.lower()=='yes':
            timeout_glob*=2
            return adaptive_scraping(url)
        else:
            return None
    except SSLError as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('We can skip verification test. Do you want to continue?')
        ans=input('\n[Yes|No]\t')
        if ans.lower()=='yes':
            verify_glob=False
            return adaptive_scraping(url)
        else:
            return None
    except TooManyRedirects as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('We can increase the number of allowed redirects. Do you want to continue?')
        ans=input('\n[Yes|No]\t')
        if ans.lower()=='yes':
            allow_glob=True
            return adaptive_scraping(url)
        else:
            return None
    except RequestException as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('Unfortunately, we have no clue what to do.  Please try again later. ')
        return None